In [1]:
import pandas as pd
import numpy as np
import csv
import sys
import keras
from keras.preprocessing.text import one_hot 

Using TensorFlow backend.


In [2]:
word_embeddings=pd.read_csv('vocab.csv')

In [3]:
text=open('Sherelok Holmes.txt', 'r')


words=[]
for line in text:
    word=line.split()
    words=words+(word)

words_txt=[word.lower() for word in words]
print ("Before {}".format(len(words)))
words=list(set(words_txt))

word_df=pd.DataFrame(words)
word_df.to_csv('words.csv', header=False, index=False)

print ("Total unique words found {}".format(len(words)))

words_to_int=dict((c,i) for i, c in enumerate(words))


Before 104740
Total unique words found 14033
{'soon,': 0, 'steady,': 1, 'drawer.': 2, '31': 3, '“good-morning,': 4, 'red-head': 5585, 'open': 5, 'come?”': 6, 'proved.': 7, 'years.': 8, 'nobleman.': 9, 'striding': 11, 'minister': 12, 'journey.': 4649, 'remarked.': 13, 'pope’s': 14, 'luxurious': 10835, 'age': 4647, 'degraded': 15, 'montague': 16, 'habit,': 17, 'colonel!': 21, 'ashes': 20, 'bottom.”': 23, 'cub,': 22, 'consequence,': 27, '‘a': 25, 'violin': 26, 'ball.”': 28, 'coppers': 29, '’89—there': 30, 'north,”': 31, 'not.”': 32, 'squeezed': 33, 'hat—“but': 34, 'earlier': 35, 'echo,': 36, 'marbank,': 37, 'night.”': 39, 'heather': 40, 'skirts.': 41, 'seeking': 44, 'weak': 43, 'apology,': 45, 'windfall': 46, 'changes': 47, '‘eg.’': 48, 'apprenticed': 4658, 'doran,': 49, 'heads': 50, 'sickness': 51, 'blind.': 52, 'hudson': 2350, 'competition': 53, 'death': 54, 'grotesque.': 55, 'weighing': 56, 'however;': 9362, 'crime.': 57, 'little,”': 58, 'avert': 59, 'fancy.”': 60, 'opponent': 61, 'unw

In [9]:
'''from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(nb_words=5000)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

word_index = tokenizer.word_index'''

'from keras.preprocessing.text import Tokenizer\nfrom keras.preprocessing.sequence import pad_sequences\n\ntokenizer = Tokenizer(nb_words=5000)\ntokenizer.fit_on_texts(text)\nsequences = tokenizer.texts_to_sequences(text)\n\nword_index = tokenizer.word_index'

In [4]:
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [5]:
embedding_matrix = np.zeros((len(words_to_int) + 1, 100))
for i, word in enumerate(words):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [6]:
#embedding_matrix has all the words in the sherelok holmes text mapped to the glove embedding matrix
embedding_matrix.shape

(14034, 100)

In [7]:
words_to_int['the']

657

In [8]:
#Making sequneces of words of length 5 to be fed into our Model
seq_length=5
X=[]
y=[]

for i in range(0, len(words)-seq_length):
    seq_in=words_txt[i:i+seq_length]
    seq_out=words_txt[i+seq_length]
    X.append([word for word in seq_in])
    y.append(seq_out)

In [9]:
#Same sequence as above with indexed as numbers
seq_length=5
X_num=[]
y_num=[]

for i in range(0, len(words)-seq_length):
    seq_in=words_txt[i:i+seq_length]
    seq_in=[words_to_int[k] for k in seq_in]
    seq_out=words_txt[i+seq_length]
    seq_out=words_to_int[seq_out]
    X_num.append([word for word in seq_in])
    y_num.append(seq_out)

In [11]:
seq_embedding=np.zeros([len(X_num),5,100],dtype=float)


In [12]:
#seq_embedding=np.zeros([len(X_num),5,100],dtype=float)
for i in range(0,len(X_num)):
    for j in range(0,len(X_num[0])):
        seq_embedding[i][j]=embedding_matrix[X_num[i][j]]

In [13]:
y_num=keras.utils.np_utils.to_categorical(y_num,nb_classes=14031)

In [14]:
train_input=seq_embedding[0:12000]
test_input=seq_embedding[12000:]
train_output=y_num[0:12000]
test_output=y_num[12000:]

In [19]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda,LSTM,Embedding
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Merge


In [20]:
model=Sequential()

In [21]:
model.add(LSTM(output_dim=32,return_sequences=False,go_backwards=False,init='he_normal',input_shape=(5,100)))

In [22]:
model.add(Dense(14031,activation='softmax'))

In [23]:
model.compile(optimizer='rmsprop',loss=keras.objectives.categorical_crossentropy,metrics=['accuracy'])

In [24]:
model.output_shape

(None, 14031)

In [25]:
model.fit(train_input,train_output,batch_size=128,nb_epoch=2,validation_data=[test_input,test_output])

Train on 12000 samples, validate on 2028 samples
Epoch 1/2
12000/12000 [==============================] - 34s - loss: 8.1260 - acc: 0.0440 - val_loss: 7.1363 - val_acc: 0.0385